# Zonal energy spectrum - ERA5

* This notebook shows how zonal energy spectrum was computed for kinetic energy and potential temperature energy. For the latter, the 500 hPa potential temperature was approximated using $\theta=T(1000/500)^{(287.0/1004)}$.

* The notebook runs with `verif_config_6h.yml` in the same folder.

In [1]:
import os
import sys
import yaml
from glob import glob
from datetime import datetime, timedelta

import numpy as np
import xarray as xr

sys.path.insert(0, os.path.realpath('../libs/'))
import verif_utils as vu
import score_utils as su

### Load config

In [2]:
config_name = os.path.realpath('verif_config.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

### ZES compute

In [4]:
variables_levels = {
    'T': [500,],
    'U': [500],
    'V': [500]
}

In [7]:
path_verif = conf['ERA5_weatherbench']['save_loc_clim']+'combined_zes_clim.nc'

# ---------------------------------------------------------------------------------------- #
# ERA5
filename_ERA5 = sorted(glob(conf['ERA5_ours']['save_loc']))

# pick years (use IFS years)
year_range = conf['IFS']['year_range']
years_pick = np.arange(year_range[0], year_range[1]+1, 1).astype(str)
filename_ERA5 = [fn for fn in filename_ERA5 if any(year in fn for year in years_pick)]

# merge yearly ERA5 as one
ds_ERA5 = [vu.get_forward_data(fn) for fn in filename_ERA5]
ds_ERA5_merge = xr.concat(ds_ERA5, dim='time')

# subset merged ERA5 and unify coord names
ds_ERA5_merge = vu.ds_subset_everything(ds_ERA5_merge, variables_levels)

L_time = len(ds_ERA5_merge['time'])

verif_results = []

for i_time in range(L_time):
    ds_ERA5 = ds_ERA5_merge.isel(time=i_time)    
    ds_ERA5 = ds_ERA5.load()

    # -------------------------------------------------------------- #
    # potential temperature
    ds_ERA5['theta'] = ds_ERA5['T'] * (1000/500)**(287.0/1004)

    # -------------------------------------------------------------- #
    zes_temp = []
    for var in ['U', 'V', 'theta']:
        zes = su.zonal_energy_spectrum_sph(ds_ERA5.isel(latitude=slice(1, None)), var)
        zes_temp.append(zes)
        
    verif_results.append(xr.merge(zes_temp))

ds_verif = xr.concat(verif_results, dim='time')

In [9]:
path_verif

'/glade/campaign/cisl/aiml/ksha/CREDIT_physics/VERIF/ERA5_clim/combined_zes_clim.nc'

In [10]:
#ds_verif.to_netcdf(path_verif)